In [1]:
import astropy.units as u
import astropy.constants as c

from ipynb.fs.full.Wind_accretion import *

In [2]:
def hasHotComp_mask_G_WD_unstable(Data, mask_G_WD):

    SWs = Data['BSE_Switch_Log']  # Access the switch log data
        
    # Very sketchy but I make the last one False because I have no way to calculate its lifetime
    mask_G_WD[-1] = False
    
    mask_plus_one = [False] * len(mask_G_WD)
    for i in range(len(mask_G_WD)):
        if mask_G_WD[i]:
            
            mask_plus_one[i + 1] = True
    
    t_GiantStage  = np.array(SWs['Time'][()][mask_plus_one] - SWs['Time'][()][mask_G_WD]) * u.Myr
        
    # A better way to do the next thing is (probably) to control that the Seed of the next switch-state is the same
    t_GiantStage[t_GiantStage < 0] = 0
    
    # Stellar properties
    Type_G         = SWs['Stellar_Type(1)'][()][mask_G_WD]
    Type_WD        = SWs['Stellar_Type(2)'][()][mask_G_WD]
    Mass_G         = SWs['Mass(1)'][()][mask_G_WD]         * u.Msun
    Mass_WD        = SWs['Mass(2)'][()][mask_G_WD]         * u.Msun
    Radius_G       = SWs['Radius(1)'][()][mask_G_WD]       * u.Rsun
    Radius_WD      = SWs['Radius(2)'][()][mask_G_WD]       * u.Rsun
    Luminosity_G   = SWs['Luminosity(1)'][()][mask_G_WD]   * u.Lsun
    Luminosity_WD  = SWs['Luminosity(2)'][()][mask_G_WD]   * u.Lsun
        
    # Binary properties
    a      = SWs['SemiMajorAxis'][()][mask_G_WD] * u.Rsun
    
    # v_orb is not implemented in COMPAS (anymore)
    # If this is used more than just here, consider making it a function
    v_orb = np.sqrt( c.G * (Mass_G + Mass_WD) / a ).to(u.km / u.s)

        
    #____________________________________________________________________
        
    # Maximum masstransfer during the stage
        
    t_Gs = np.linspace(0, t_GiantStage, 100)
    dt   = t_GiantStage/(101)
                        
    t_visible = np.zeros(len(Mass_WD)) * u.Myr
        
    Delta_M = np.zeros(len(Mass_WD)) * u.Msun # H built up on the surface of the WD
    
    t_on = np.zeros(len(Mass_WD)) * u.Myr
    t_cool = np.zeros(len(Mass_WD)) * u.Myr

    L_on = np.zeros(len(Mass_WD)) * u.Lsun
    
    M_crit = ign_mass(Mass_WD)
    
    Mdot = interpWindLossALL_t(Type_G, Mass_G, t_Gs)
        
    #____________________________________________________________________        

    for t_ind in range(len(t_Gs)):
                
        t = t_Gs[t_ind]
                
        # be careful about what should be done with Mdot_accum and what should be done with Mdot_acc
        Mdotacc = Mdot_acc(Mass_WD, a, v_orb, Mdot[:,t_ind], Radius_G, Mass_G)
        
        Mdotaccum = Mdot_accum(Mdotacc, Mass_WD)

        Delta_M += Mdotaccum * dt
                
        # could we combine the masks such that I only check the stable datas for values that can be stable?
        # would this even be quicker considering I calculate the Mdotacc either way?
        
        stableSyS = isStable(Mass_WD, Mdotacc)
        
        startSyNe    = (Delta_M > M_crit)
        
        startBurning = (stableSyS | startSyNe)
        stillBurning = (t < t_cool)
        
        isBurning    = startBurning | stillBurning
        isSyNe       = isBurning & ~stableSyS
        
        L_on[startBurning] = L_Sy(Mass_WD[startBurning], Radius_WD[startBurning], Mdotaccum[startBurning]) + Luminosity_WD[startBurning]
        
        t_on[startBurning]   = t[startBurning] + t_on_fun(Mdotacc[startBurning], Mass_WD[startBurning], Radius_WD[startBurning]) # t in Myr
        t_cool[startBurning] = t_on[startBurning] + t_cool_fun(Mass_WD[startBurning]) # t in Myr
        
        # L_on[t > t_cool] = Luminosity_WD[t > t_cool]
        L_on[t > t_cool] = 0 # this ignores systems that have no SyStable or SyNe 

        visible = (L_on > 10 * u.Lsun)
        
        Delta_M[isBurning] = 0
        
        t_visible[isSyNe & visible] += np.min([dt[isSyNe & visible],t_cool[isSyNe & visible] - t[isSyNe & visible]],0) * u.Myr
            
    return t_visible   

In [3]:
def hasHotComp_mask_WD_G_unstable(Data, mask_WD_G):

    SWs = Data['BSE_Switch_Log']  # Access the switch log data
        
    # Very sketchy but I make the last one False because I have no way to calculate its lifetime
    mask_WD_G[-1] = False
    
    mask_plus_one = [False] * len(mask_WD_G)
    for i in range(len(mask_WD_G)):
        if mask_WD_G[i]:
            
            mask_plus_one[i + 1] = True
    
    t_GiantStage  = np.array(SWs['Time'][()][mask_plus_one] - SWs['Time'][()][mask_WD_G]) * u.Myr
        
    # A better way to do the next thing is (probably) to control that the Seed of the next switch-state is the same
    t_GiantStage[t_GiantStage < 0] = 0
    
    # Stellar properties
    Type_WD        = SWs['Stellar_Type(1)'][()][mask_WD_G]
    Type_G         = SWs['Stellar_Type(2)'][()][mask_WD_G]
    Mass_WD        = SWs['Mass(1)'][()][mask_WD_G]         * u.Msun
    Mass_G         = SWs['Mass(2)'][()][mask_WD_G]         * u.Msun
    Radius_WD      = SWs['Radius(1)'][()][mask_WD_G]       * u.Rsun
    Radius_G       = SWs['Radius(2)'][()][mask_WD_G]       * u.Rsun
    Luminosity_WD  = SWs['Luminosity(1)'][()][mask_WD_G]   * u.Lsun
    Luminosity_G   = SWs['Luminosity(2)'][()][mask_WD_G]   * u.Lsun
        
    # Binary properties
    a      = SWs['SemiMajorAxis'][()][mask_WD_G] * u.Rsun
    
    # v_orb is not implemented in COMPAS (anymore)
    # If this is used more than just here, consider making it a function
    v_orb = np.sqrt( c.G * (Mass_G + Mass_WD) / a ).to(u.km / u.s)

        
    #____________________________________________________________________
        
    # Maximum masstransfer during the stage
        
    t_Gs = np.linspace(0, t_GiantStage, 100)
    dt   = t_GiantStage/(101)
                        
    t_visible = np.zeros(len(Mass_WD)) * u.Myr
        
    Delta_M = np.zeros(len(Mass_WD)) * u.Msun # H built up on the surface of the WD
    
    t_on = np.zeros(len(Mass_WD)) * u.Myr
    t_cool = np.zeros(len(Mass_WD)) * u.Myr

    L_on = np.zeros(len(Mass_WD)) * u.Lsun
    
    M_crit = ign_mass(Mass_WD).to(u.Msun)
    
    Mdot = interpWindLossALL_t(Type_G, Mass_G, t_Gs)
        
    #____________________________________________________________________        

    for t_ind in range(len(t_Gs)):
                
        t = t_Gs[t_ind]
                
        # be careful about what should be done with Mdot_accum and what should be done with Mdot_acc
        Mdotacc = Mdot_acc(Mass_WD, a, v_orb, Mdot[:,t_ind], Radius_G, Mass_G).to(u.Msun / u.yr)
        
        Mdotaccum = Mdot_accum(Mdotacc, Mass_WD).to(u.Msun / u.yr)

        Delta_M += (Mdotaccum * dt).to(u.Msun)
                
        # could we combine the masks such that I only check the stable datas for values that can be stable?
        # would this even be quicker considering I calculate the Mdotacc either way?
                
        stableSyS = isStable(Mass_WD, Mdotacc)
            
        startSyNe    = (Delta_M > M_crit)
        
        startBurning = (stableSyS | startSyNe)
        stillBurning = (t < t_cool)
        
        isBurning    = startBurning | stillBurning
        isSyNe       = isBurning & ~stableSyS
        
        L_on[startBurning] = L_Sy(Mass_WD[startBurning], Radius_WD[startBurning], Mdotaccum[startBurning]) + Luminosity_WD[startBurning]
        
        t_on[startBurning]   = t[startBurning] + t_on_fun(Mdotacc[startBurning], Mass_WD[startBurning], Radius_WD[startBurning]) # t in Myr
        t_cool[startBurning] = t_on[startBurning] + t_cool_fun(Mass_WD[startBurning]) # t in Myr
        
        # L_on[t > t_cool] = Luminosity_WD[t > t_cool]
        L_on[t > t_cool] = 0 # this ignores systems that have no SyStable or SyNe 

        visible = (L_on > 10 * u.Lsun)
        
        Delta_M[isBurning] = 0
        
        t_visible[isSyNe & visible] += np.min([dt[isSyNe & visible],t_cool[isSyNe & visible] - t[isSyNe & visible]],0) * u.Myr
            
    return t_visible   